In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import json
import pickle
# import tensorflow as tf
import keras
import keras.backend as K
# from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# import keras_retinanet
# import keras_retinanet.bin.train
from sklearn.utils import shuffle 
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [3]:
json_file = open('/home/jovyan/hico_model.json', 'r')
model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(model_json)
model.load_weights('/home/jovyan/hico_second_try.h5')

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, None, 2, 4)   0           input_8[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 800, 1200, 3) 0                                            
__________

In [5]:
test_data = pd.read_json('../test_data_with_obj_id.json')
test_data = test_data.sort_index()
retinanet_test_data = pd.read_json('retinanet_test_data.json')
retinanet_test_data = retinanet_test_data.sort_index()
with open('../hoi_dict.json', 'r') as f:
    hoi_dict = json.load(f)
with open('../obj_dict.json', 'r') as f:
    obj_dict = json.load(f)
obj_word_to_id = {v:int(k) for k,v in obj_dict.items()}

In [6]:
def test_data_gen(retinanet_test_data,test_data,id_):
    img_stack = np.array([]).reshape(0,800,1200,3)
    human_stack = np.array([]).reshape(0,4)
    object_stack = np.array([]).reshape(0,4)
    object_class_stack = np.array([]).reshape(1,0)
            #image
    img = cv2.imread(os.path.join('/home/jovyan/projectdata/cht01/hico_20160224_det/images/test2015/',test_data['name'][id_]))
    img = cv2.resize(img, (1200,800))
    img = img/255
    img = np.expand_dims(img,axis=0)
    img_stack = np.row_stack([img_stack,img])

            
            #human object bbox pair & object class
    for i in range(len(retinanet_test_data.human_boxes[id_])):
        if (np.max(retinanet_test_data.human_scores[id_]) >0.5) & (retinanet_test_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_test_data.human_boxes[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
#         ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
        human_stack = np.row_stack([human_stack,ho_pair_h])

    for j in range(len(retinanet_test_data.obj_boxes[id_])):
        if (np.max(retinanet_test_data.obj_scores[id_]) >0.5) & (retinanet_test_data.obj_scores[id_][j]<0.5):
            continue
#         print(j)
        ho_pair_o = retinanet_test_data.obj_boxes[id_][j]
        object_class = retinanet_test_data.obj_classes[id_][j]
        ho_pair_o = np.expand_dims(ho_pair_o,axis=0)
#       ho_pair_o = np.expand_dims(ho_pair_o,axis=0)

#       object_class = np.expand_dims(object_class,axis=0)
            
#       img_stack = np.row_stack([img_stack,img])
        object_stack = np.row_stack([object_stack,ho_pair_o])
        object_class_stack = np.column_stack([object_class_stack,object_class])
#     img_stack = np.expand_dims(img_stack,axis=0)
    human_stack = np.expand_dims(human_stack,axis=0)
    object_stack = np.expand_dims(object_stack,axis=0)
#     object_class_stack = np.expand_dims(object_class_stack,axis=0)
    test_batch = [img_stack,human_stack,object_stack,object_class_stack]
    return test_batch

In [9]:
test_one_img = test_data_gen(retinanet_test_data,test_data,1)
test_one_img[0].shape,test_one_img[1].shape,test_one_img[2].shape,test_one_img[3].shape

((1, 800, 1200, 3), (1, 4, 4), (1, 5, 4), (1, 5))

In [10]:
pred_all = []
for i in tqdm(range(test_data.shape[0])):
    test_one_img = test_data_gen(retinanet_test_data,test_data,i)
    if test_one_img[1].any()==True:
        pred = model.predict(test_one_img)
    else:
        pred = [np.array([]).reshape((1,0,600)),np.array([]).reshape((1,0))]
    pred_all.append(pred)

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

In [11]:
with open('predict_result','wb') as file:
    pickle.dump(pred_all,file)

# testing visualization

In [6]:
with open('predict_result','rb') as file:
    pred_all = pickle.load(file)
test_data = pd.read_json('../test_data_with_obj_id.json')
test_data = test_data.sort_index()
retinanet_test_data = pd.read_json('retinanet_test_data.json')
retinanet_test_data = retinanet_test_data.sort_index()
with open('../hoi_dict.json', 'r') as f:
    hoi_dict = json.load(f)
with open('../obj_dict.json', 'r') as f:
    obj_dict = json.load(f)
obj_word_to_id = {v:int(k) for k,v in obj_dict.items()}

In [7]:
def visual_data_gen(retinanet_test_data,test_data,id_):
    img_stack = np.array([]).reshape(0,800,1200,3)
    human_stack = np.array([]).reshape(0,4)
    object_stack = np.array([]).reshape(0,4)
    object_class_stack = np.array([]).reshape(1,0)
            #image
    img = cv2.imread(os.path.join('/home/jovyan/projectdata/cht01/hico_20160224_det/images/test2015/',test_data['name'][id_]))
    img = cv2.resize(img, (1200,800))
    img = img/255
    img = np.expand_dims(img,axis=0)
    img_stack = np.row_stack([img_stack,img])

            
            #human object bbox pair & object class
    for i in range(len(retinanet_test_data.human_boxes[id_])):
        if (np.max(retinanet_test_data.human_scores[id_]) >0.5) & (retinanet_test_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_test_data.human_boxes[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
        human_stack = np.row_stack([human_stack,ho_pair_h])

    for j in range(len(retinanet_test_data.obj_boxes[id_])):
        if (np.max(retinanet_test_data.obj_scores[id_]) >0.5) & (retinanet_test_data.obj_scores[id_][j]<0.5):
            continue
        ho_pair_o = retinanet_test_data.obj_boxes[id_][j]
        object_class = retinanet_test_data.obj_classes[id_][j]
        ho_pair_o = np.expand_dims(ho_pair_o,axis=0)

        object_stack = np.row_stack([object_stack,ho_pair_o])
        object_class_stack = np.column_stack([object_class_stack,object_class])
    human_stack = np.expand_dims(human_stack,axis=0)
    object_stack = np.expand_dims(object_stack,axis=0)
    test_batch = [img_stack,human_stack,object_stack,object_class_stack]
    pred = model.predict(test_batch)
    return img[0],human_stack[0],object_stack[0],pred[0][0]

In [ ]:
a,b,c,d = visual_data_gen(retinanet_test_data,test_data,9)
# RGB_img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
RGB_img = a[:,:,::-1]
action = np.where(d>0.7)[1]
for i in action:
    print(hoi_dict[str(i+1)])
plt.imshow(RGB_img)
# b = b.astype('int64')
# c = c.astype('int64')
# fig,(ax1,ax2) = plt.subplots(1,2)
# fig.set_size_inches(16,8)
# for i in b:
#     cv2.rectangle(RGB_img, (i[0],i[1]),(i[2],i[3]), (0.5,0,0.5), 2)
# ax1.imshow(RGB_img)
# for i in c:
#     cv2.rectangle(RGB_img, (i[0],i[1]),(i[2],i[3]), (0,0.5,0), 2)
# ax2.imshow(RGB_img)

# mAP calculation

In [8]:
with open('predict_result','rb') as file:
    pred_all = pickle.load(file)
test_data = pd.read_json('../test_data_with_obj_id.json')
test_data = test_data.sort_index()
retinanet_test_data = pd.read_json('retinanet_test_data.json')
retinanet_test_data = retinanet_test_data.sort_index()
with open('../hoi_dict.json', 'r') as f:
    hoi_dict = json.load(f)
with open('../obj_dict.json', 'r') as f:
    obj_dict = json.load(f)
obj_word_to_id = {v:int(k) for k,v in obj_dict.items()}

In [9]:
#先跑object loop再跑human loop
#bbox = (x1,y1,x2,y2)
def bbox_gen(retinanet_test_data,id_):
    human_stack = np.array([]).reshape(0,4)
    object_stack = np.array([]).reshape(0,4)
    object_class_stack = np.array([]).reshape(0,1)

    for i in range(len(retinanet_test_data.human_boxes_scale[id_])):
        if (np.max(retinanet_test_data.human_scores[id_]) >0.5) & (retinanet_test_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_test_data.human_boxes_scale[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)

        for j in range(len(retinanet_test_data.obj_boxes_scale[id_])):
            if (np.max(retinanet_test_data.obj_scores[id_]) >0.5) & (retinanet_test_data.obj_scores[id_][j]<0.5):
                continue
            ho_pair_o = retinanet_test_data.obj_boxes_scale[id_][j]
            ho_pair_o = np.expand_dims(ho_pair_o,axis=0)
            
            object_class = retinanet_test_data.obj_classes[id_][j]
            object_class = np.expand_dims(object_class,axis=0)
            
            human_stack = np.row_stack([human_stack,ho_pair_h])
            object_stack = np.row_stack([object_stack,ho_pair_o])
            object_class_stack = np.row_stack([object_class_stack,object_class])
    object_class_stack = np.squeeze(object_class_stack)
    
    ho_bbox = [human_stack,object_stack,object_class_stack]
    return ho_bbox

In [ ]:
#calc all image per action for AP
AP_list = []
for action_id in tqdm(range(1,601)):
    bool_list = [item[1]==action_id for item in gt_action_all]
    gt_1action_action = [item for i,item in enumerate(gt_action_all) if bool_list[i]==True] 
    gt_1action_h_bbox = [item for i,item in enumerate(gt_h_bbox_all) if bool_list[i]==True] 
    gt_1action_o_bbox = [item for i,item in enumerate(gt_o_bbox_all) if bool_list[i]==True] 

    tp = np.zeros([len(result_h_bbox_all)])
    fp = np.zeros([len(result_h_bbox_all)])
    
    npos = len(gt_1action_action)
    result_1action_score = [(item[0],item[1][action_id-1]) for item in result_action_all]
    indices = np.argsort(result_1action_score,axis=0)[:,1][::-1]
    score = [result_1action_score[i] for i in indices]
    h_bbox = [result_h_bbox_all[i] for i in indices]
    o_bbox = [result_o_bbox_all[i] for i in indices]

    for i in tqdmrange(len(h_bbox)):
        for j in range(len(gt_1action_h_bbox)):
            if h_bbox[i][0]==gt_1action_h_bbox[j][0]:
                iou_h = bb_iou(h_bbox[i][1],gt_1action_h_bbox[j][1])
                iou_o = bb_iou(o_bbox[i][1],gt_1action_o_bbox[j][1])
                if np.min([iou_h,iou_o])>0.5:
                    tp[i]=1
                else:
                    fp[i]=1
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp/npos
    prec = tp/(fp+tp)
    AP=0
    for t in np.arange(0.0, 1.0, 0.1):
        if prec[rec>=t].any():
            p = np.max(prec[rec>=t])
            AP = AP +p/11
        else:
            p=0
            AP = AP +p/11
    AP_list.append(AP)

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

In [11]:
#bbox = (x1,x2,y1,y2) >>> (x1,y1,x2,y2)
gt_action = []
gt_h_bbox = []
gt_o_bbox = []
gt_o_class = []
for id_ in tqdm(range(len(test_data))):
    gt_action_sublist = []
    gt_h_bbox_sublist = []
    gt_o_bbox_sublist = []
    gt_o_class_sublist = []  
    h_box = np.array([[item[0],item[2],item[1],item[3]] for item in np.array(test_data.human_bbox[id_])])
    o_box = np.array([[item[0],item[2],item[1],item[3]] for item in np.array(test_data.object_bbox[id_])])
    pair_no = np.array(test_data.pair_no[id_])
    action_no = np.array(test_data.action_no[id_])
    obj_id = np.array(test_data.obj_id[id_])
    max_o_box_id = 0
    last_h_box_id = 0
    last_o_box_id = 0
    h_box_id_shift = 0
    o_box_id_shift = 0
    action_id = 0
    if pair_no.any():
        for j in pair_no:
            if len(j)==2:
                h_box_id = j[0] + h_box_id_shift 
                o_box_id = j[1] + o_box_id_shift
                if h_box_id > last_h_box_id or o_box_id > last_o_box_id:
                    gt_action_sublist.append(action_no[action_id])
                    gt_o_class_sublist.append(obj_word_to_id[hoi_dict[str(action_no[action_id])][1].replace('_'," ")])
                    gt_h_bbox_sublist.append(h_box[h_box_id-1])
                    gt_o_bbox_sublist.append(o_box[o_box_id-1])
                    last_h_box_id = h_box_id
                    last_o_box_id = o_box_id
                    if o_box_id>max_o_box_id:
                        max_o_box_id=o_box_id
                else:
                    h_box_id_shift = last_h_box_id
                    o_box_id_shift = max_o_box_id
                    action_id +=1
                    h_box_id = j[0] + h_box_id_shift 
                    o_box_id = j[1] + o_box_id_shift
                    gt_action_sublist.append(action_no[action_id])
                    gt_o_class_sublist.append(obj_word_to_id[hoi_dict[str(action_no[action_id])][1].replace('_'," ")])
                    gt_h_bbox_sublist.append(h_box[h_box_id-1])
                    gt_o_bbox_sublist.append(o_box[o_box_id-1])
                    last_h_box_id = h_box_id
                    last_o_box_id = o_box_id
                    if o_box_id>max_o_box_id:
                        max_o_box_id=o_box_id
        gt_action.append(np.array(gt_action_sublist))
        gt_h_bbox.append(np.array(gt_h_bbox_sublist))
        gt_o_bbox.append(np.array(gt_o_bbox_sublist))
        gt_o_class.append(np.array(gt_o_class_sublist))
    else:
#         print(id_)
        gt_action.append(np.array(gt_action_sublist))
        gt_h_bbox.append(np.array(gt_h_bbox_sublist))
        gt_o_bbox.append(np.array(gt_o_bbox_sublist))
        gt_o_class.append(np.array(gt_o_class_sublist))

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

In [13]:
# (x1, y1, x2, y2)
def bb_iou(boxA,boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
    if boxA[0]>boxB[2] or boxA[2]<boxB[0] or boxA[1]>boxB[3] or boxA[3]<boxB[1]:
        iou = 0
    else:
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        # compute the area of intersection rectangle
        interArea = (xB - xA + 1) * (yB - yA + 1)
 
        # compute the area of both the prediction and ground-truth
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [19]:
#reference from Mask RCNN mAP
AP_list = []
for id_ in tqdm(range(len(result_h_bbox))):
# for id_ in tqdm(range(1,2)):   
    AP_sublist = []
    for action_id in gt_action[id_]:
        gt_1action_h_bbox = gt_h_bbox[id_][gt_action[id_]==action_id]
        gt_1action_o_bbox = gt_o_bbox[id_][gt_action[id_]==action_id]
        gt_1action_action = gt_action[id_][gt_action[id_]==action_id]
        gt_match = np.zeros([gt_1action_h_bbox.shape[0]])
        pred_match = np.zeros([result_h_bbox[id_].shape[0]])
        indices = np.argsort(result_action[id_][:,action_id-1])[::-1]
        score = result_action[id_][:,action_id-1][indices]
        h_bbox = result_h_bbox[id_][indices]
        o_bbox = result_o_bbox[id_][indices]
        for i in range(len(h_bbox)):
            for j in range(len(gt_1action_h_bbox)):
                iou_h = bb_iou(h_bbox[i],gt_1action_h_bbox[j])
                iou_o = bb_iou(o_bbox[i],gt_1action_o_bbox[j])
                if np.min([iou_h,iou_o])>0.5:
                    if gt_match[j]==1:
                        continue
                    pred_match[i]=1
                    gt_match[j]=1
        precisions = np.cumsum(pred_match) / (np.arange(len(pred_match)) + 1)
        recalls = np.cumsum(pred_match).astype(np.float32) / len(gt_match)
        precisions = np.concatenate([[0], precisions, [0]])
        recalls = np.concatenate([[0], recalls, [1]])

        for i in range(len(precisions) - 2, -1, -1):
            precisions[i] = np.maximum(precisions[i], precisions[i + 1])
        indice = np.where(recalls[:-1] != recalls[1:])[0] + 1
        AP = ((recalls[indice] - recalls[indice - 1]) *precisions[indice])[0]
        AP_sublist.append(AP)
    AP_list.append(AP_sublist)

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

In [20]:
AP_list = np.array(AP_list)
temp = [item for sublist in AP_list for item in sublist]
temp = np.array(temp)
temp[np.isnan(temp)]=0
np.mean(temp)

0.17916038174882379

In [21]:
##reference from hico-det matlab code
AP_list = []
for id_ in tqdm(range(len(result_h_bbox))):   
# for id_ in tqdm(range(1,2)):   

    AP_sublist = []
    for action_id in gt_action[id_]:
        gt_1action_h_bbox = gt_h_bbox[id_][gt_action[id_]==action_id]
        gt_1action_o_bbox = gt_o_bbox[id_][gt_action[id_]==action_id]
        gt_1action_action = gt_action[id_][gt_action[id_]==action_id]
        tp = np.zeros([result_h_bbox[id_].shape[0]])
        fp = np.zeros([result_h_bbox[id_].shape[0]])
        npos = gt_1action_action.shape[0]
        indices = np.argsort(result_action[id_][:,action_id-1])[::-1]
        score = result_action[id_][:,action_id-1][indices]
        h_bbox = result_h_bbox[id_][indices]
        o_bbox = result_o_bbox[id_][indices]
        for i in range(len(h_bbox)):
            for j in range(len(gt_1action_h_bbox)):
                iou_h = bb_iou(h_bbox[i],gt_1action_h_bbox[j])
                iou_o = bb_iou(o_bbox[i],gt_1action_o_bbox[j])
                if np.min([iou_h,iou_o])>0.5:
                    tp[i]=1
                else:
                    fp[i]=1
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp/npos
        prec = tp/(fp+tp)
        AP=0
        for t in np.arange(0.0, 1.0, 0.1):
            if prec[rec>=t].any():
                p = np.max(prec[rec>=t])
                AP = AP +p/11
            else:
                p=0
                AP = AP +p/11
        AP_sublist.append(AP)
    AP_list.append(AP_sublist)

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

In [22]:
AP_list = np.array(AP_list)
temp = [item for sublist in AP_list for item in sublist]
temp = np.array(temp)
temp[np.isnan(temp)]=0
np.mean(temp)

0.17633376885747484

# possibly the right way

In [14]:
gt_action_all = [(i,item) for i,sublist in enumerate(gt_action) for item in sublist]
gt_h_bbox_all = [(i,item) for i,sublist in enumerate(gt_h_bbox) for item in sublist]
gt_o_bbox_all = [(i,item) for i,sublist in enumerate(gt_o_bbox) for item in sublist]
result_action_all = [(i,item) for i,sublist in enumerate(result_action) for item in sublist]
result_h_bbox_all = [(i,item) for i,sublist in enumerate(result_h_bbox) for item in sublist]
result_o_bbox_all = [(i,item) for i,sublist in enumerate(result_o_bbox) for item in sublist]

In [34]:
#calc all image per action for AP
AP_list = []
for action_id in tqdm(range(1,601)):
# for action_id in tqdm(range(1,2)):

    bool_list = [item[1]==action_id for item in gt_action_all]
    gt_1action_action = [item for i,item in enumerate(gt_action_all) if bool_list[i]==True] 
    gt_1action_h_bbox = [item for i,item in enumerate(gt_h_bbox_all) if bool_list[i]==True] 
    gt_1action_o_bbox = [item for i,item in enumerate(gt_o_bbox_all) if bool_list[i]==True] 

    tp = np.zeros([len(result_h_bbox_all)])
    fp = np.zeros([len(result_h_bbox_all)])
    
    npos = len(gt_1action_action)
    result_1action_score = [(item[0],item[1][action_id-1]) for item in result_action_all]
    indices = np.argsort(result_1action_score,axis=0)[:,1][::-1]
    score = [result_1action_score[i] for i in indices]
    h_bbox = [result_h_bbox_all[i] for i in indices]
    o_bbox = [result_o_bbox_all[i] for i in indices]

    for i in range(len(h_bbox)):
        for j in range(len(gt_1action_h_bbox)):
            if h_bbox[i][0]==gt_1action_h_bbox[j][0]:
                iou_h = bb_iou(h_bbox[i][1],gt_1action_h_bbox[j][1])
                iou_o = bb_iou(o_bbox[i][1],gt_1action_o_bbox[j][1])
                if np.min([iou_h,iou_o])>0.5:
                    tp[i]=1
                else:
                    fp[i]=1
            else:
                fp[i]=1
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp/npos
    prec = tp/(fp+tp)
    AP=0
    for t in np.arange(0.0, 1.0, 0.1):
        if prec[rec>=t].any():
            p = np.max(prec[rec>=t])
            AP = AP +p/11
        else:
            p=0
            AP = AP +p/11
    AP_list.append(AP)

HBox(children=(IntProgress(value=0, max=600), HTML(value='')))

In [35]:
AP_list = np.array(AP_list)
np.mean(AP_list)

0.004247795305763905

In [37]:
np.max(AP_list)

0.11911104338220219